# Lab 5: Unified Batch & Streaming - Same Code

## 🎯 **Learning Objectives:**
- Write shared transformation functions
- Apply same code cho batch và streaming
- Process historical data với same logic
- Write to same Iceberg tables
- Compare results và performance

## 📚 **Key Concepts:**
1. **Unified Code**: Same transformation cho batch và streaming
2. **Code Reuse**: Write once, use for both
3. **Same Storage**: Cùng Iceberg tables
4. **Consistency**: Same logic → same results


In [ ]:
# Setup
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("UnifiedBatchStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark Session initialized!")


## Exercise 1: Shared Transformation Function

### Key Idea:
Viết 1 function dùng cho cả batch và streaming.


In [ ]:
# Shared Transformation Function
print("🔧 Exercise 1: Shared Transformation Function")
print("=" * 60)

def process_stock_trades(df):
    """
    Process stock trades - works for both batch and streaming!
    Same logic, different triggers.
    """
    return (df
        .filter(col("price") > 0)
        .withColumn("total_value", col("price") * col("volume"))
        .withColumn("symbol", upper(trim(col("symbol"))))
        .groupBy("symbol", window(col("event_timestamp"), "1 minute"))
        .agg(
            avg("price").alias("avg_price"),
            sum("volume").alias("total_volume"),
            sum("total_value").alias("total_value"),
            count("*").alias("trade_count")
        )
        .select(
            col("window.start").alias("window_start"),
            col("window.end").alias("window_end"),
            col("symbol"),
            col("avg_price"),
            col("total_volume"),
            col("total_value"),
            col("trade_count")
        )
    )

print("✅ Shared function defined!")
print("\n💡 This function works for:")
print("   ✅ Streaming DataFrames (readStream)")
print("   ✅ Batch DataFrames (read)")
print("   ✅ Same logic, same results!")


## Exercise 2: Apply to Streaming

### Streaming: Real-time processing từ Kafka


In [ ]:
# Apply to Streaming
print("📡 Exercise 2: Apply to Streaming")
print("=" * 60)

# Read from Kafka (streaming)
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "stock-trades") \
    .option("startingOffsets", "earliest") \
    .load()

# Parse JSON (simplified - use actual schema)
# ... parsing code ...

# Apply shared function
# stream_result = process_stock_trades(parsed_stream)

print("✅ Streaming pipeline with shared function!")
print("   Same function, streaming trigger")


## Exercise 3: Apply to Batch (Same Code!)

### Batch: Historical data processing
### Same function, same results!


In [ ]:
# Apply to Batch (Same Code!)
print("📦 Exercise 3: Apply to Batch (Same Code!)")
print("=" * 60)

# Read historical data (batch)
# batch_df = spark.read.parquet("/historical/trades/")

# Apply SAME function!
# batch_result = process_stock_trades(batch_df)

print("✅ Batch pipeline with SAME function!")
print("   Same function, batch trigger")
print("   Same logic → Same results!")

print("\n💡 Key Benefits:")
print("   ✅ Code reuse: Write once, use twice")
print("   ✅ Consistency: Same logic → same results")
print("   ✅ Maintainability: Only 1 codebase to maintain")
print("   ✅ Testing: Test 1 function for both use cases")


## Summary

### ✅ Key Takeaways:
1. **Unified Code**: Same function cho batch và streaming
2. **Code Reuse**: Write once, use for both
3. **Consistency**: Same logic → same results
4. **Maintainability**: 1 codebase instead of 2
5. **This is the power of Streaming Lakehouse!**

### 🎯 Why This Matters:
- **Lambda**: Would need 2 separate functions
- **Kappa**: Only streaming, no batch
- **Streaming Lakehouse**: 1 function, both use cases ✅
